In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pytpc
from pytpc.constants import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.signal
import h5py
from scipy.interpolate import UnivariateSpline, interp1d
from scipy.optimize import differential_evolution, minimize, basinhopping, leastsq
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LogNorm
import os
import cmaps
import yaml

In [ ]:
sns.set_style()
sns.set_context('notebook')
sns.set_color_codes()

In [ ]:
import sys
sys.path.append('../fitters/')
sys.path.append('../')

In [ ]:
from find_inhib import find_exclusion_region, read_lookup_table

In [ ]:
import cfit
import atmc
import fitutils

In [ ]:
runtables = pd.read_csv('/Users/josh/Documents/Data/Meta/e15503b/e15503b-rundata-parseable.csv', index_col=0)

In [ ]:
def find_proton_params(th3, m1, m2, m3, m4, T):
    s = (m1 + m2)**2 + 2 * m2 * T
    pcm  = np.sqrt(((s - m1**2 - m2**2)**2 - 4 * m1**2 * m2**2) / (4 * s))
    ppcm = np.sqrt(((s - m3**2 - m4**2)**2 - 4 * m3**2 * m4**2) / (4 * s))
    chi = np.log((pcm + np.sqrt(m2**2 + pcm**2)) / m2)
    E3cm = np.sqrt(ppcm**2 + m3**2)
#     print(np.sqrt(s), E3cm - m3)
    
    coshx = np.cosh(chi)
    sinhx = np.sinh(chi)
    
    root = np.sqrt(coshx**2 * (E3cm**2 + m3**2 * (-coshx**2 + np.cos(th3)**2 * sinhx**2)))
    denom = ppcm * (coshx**2 - np.cos(th3)**2 * sinhx**2)
    sinthcm = np.sin(th3) * (E3cm * np.cos(th3) * sinhx + root) / denom
    p3 = ppcm * sinthcm / np.sin(th3)
    E3 = np.sqrt(p3**2 + m3**2)
    return sinthcm, E3

In [ ]:
lookup = read_lookup_table('/Users/josh/Documents/Data/Meta/e15503b/Lookup20150611.csv')
excl_pads, lowg_pads = find_exclusion_region('/Users/josh/Documents/Data/Meta/e15503b/configs/run_0066/configure-e15503b.xcfg',
                                  lookup)

In [ ]:
def transfer(amp, shape, offset=0):
    t = np.arange(512)
    f = amp*np.exp(-3*(t-offset)/shape)*np.sin((t-offset)/shape)*((t-offset)/shape)**3 / 0.044
    return np.where(t >= offset, f, np.zeros(t.shape))

# Pad sizes

In [ ]:
pads = pytpc.generate_pad_plane()

In [ ]:
bigpads = np.where(np.round(np.abs(pads[:, 1, 1] - pads[:, 0, 1])) > 6)[0]
smallpads = np.where(np.round(np.abs(pads[:, 1, 1] - pads[:, 0, 1])) < 6)[0]
assert(len(bigpads) + len(smallpads) == 10240)

# Read event from results file

In [ ]:
goodevts = pd.read_hdf('/Users/josh/Documents/Data/Results/e15503b/cfit/Feb1/results.h5', 'GoodEvtList')

In [ ]:
cfres_list = goodevts[(goodevts.run_num == 75)]

In [ ]:
# cfres = cfres_list.iloc[25] # troublesome one: iloc 4, run 75, evt 62
cfres = cfres_list[(cfres_list.run_num == 75) & (cfres_list.evt_id == 3924)].iloc[0] # troublesome one: iloc 4, run 75, evt 62
run_num = int(cfres.run_num)
evt_id = int(cfres.evt_id)

In [ ]:
run_num, evt_id

In [ ]:
print(cfres.kine_en_p, cfres.curv_en_p)
print(cfres.kine_en_c12, cfres.curv_en_c12)

In [ ]:
with open('../fitters/config_e15503b_macmini.yml', 'r') as f:
    config = yaml.load(f)
# config['vdmag'] = -2.5
# config['tilt'] = 7.0

In [ ]:
tilt = config['tilt'] * degrees # 6*degrees
clock = config['clock']
beam_center = np.array(config['beam_center']) #(-5, -86, 289)  # in uv-space
beam_en = config['beam_enu0']

efield = np.array(config['efield'])
bfield = np.array(config['bfield'])
tmat = pytpc.utilities.tilt_matrix(-tilt)
# efield_uvw = tmat.dot(efield)
# bfield_uvw = tmat.dot(bfield)

vd = np.array(config['vd'])
beampads = np.fromfile(config['beampads_path'])

In [ ]:
with h5py.File('../monte_carlo/LUT.h5', 'r') as hf:
    lt = hf['LUT'][:]

In [ ]:
padplane = atmc.PadPlane(lt, -0.280, 0.0001, -0.280, 0.0001, -108*degrees)

In [ ]:
circle_fitter = cfit.CircleFitter(config)

In [ ]:
with h5py.File('/Users/josh/Documents/Data/Merged/e15503b/peaks/run_{:04d}_peaks_bkd_subtract.h5'.format(run_num), 'r') as h5file:
    raw_xyz = h5file['/reduced_get_events/{}'.format(evt_id)][:]

In [ ]:
#raw_xyz[np.where(np.in1d(raw_xyz[:, -1], bigpads)), -2] /= 4

In [ ]:
hits = np.zeros(10240)
hits[raw_xyz[:, -1].astype('int')] = raw_xyz[:, -2]
pytpc.pad_plot(hits, scale='linear', cmap=cmaps.viridis)

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)
sigsum = np.zeros(512)
for r in raw_xyz:
    sigsum[r[2]] += r[3]
ax[0].plot(sigsum)

ax[1].plot(raw_xyz[:, 2], raw_xyz[:, 1], '.')

ax[0].set_xlim(0, 511)

In [ ]:
def preprocess(self, raw_xyz):
        # Correct for Lorentz angle and find z dimension
        # xyz = pd.DataFrame(pytpc.evtdata.calibrate(raw_xyz, self.vd, self.clock), columns=('x', 'y', 'z', 'a', 'pad'))
        xyz = pd.DataFrame(pytpc.evtdata.calibrate(raw_xyz[np.where(~np.in1d(raw_xyz[:, -1], self.beampads))],
                                                   self.vd, self.clock, tilt=0*degrees),
                           columns=('x', 'y', 'z', 'a', 'pad'))

        xyz['z'] -= self.beam_center[2]

        # Find the untilted coordinates
        tmat = pytpc.utilities.tilt_matrix(-self.tilt)
        xyz['u'], xyz['v'], xyz['w'] = np.inner(tmat, xyz[['x', 'y', 'z']])
        xyz['u'] -= self.beam_center[0]
        xyz['v'] -= self.beam_center[1]

        return xyz

In [ ]:
xyz = preprocess(circle_fitter, raw_xyz)
# xyz = pd.DataFrame(raw_xyz, columns=('x', 'y', 'z', 'a', 'pad'))
noise = np.sum((np.sqrt(np.sum((xyz.values[:, None, :3] - xyz.values[:, :3])**2, -1)) <= 40) 
               | np.eye(len(xyz), dtype='bool'), axis=0) < 3

xyz = xyz.iloc[~noise].copy().sort_values('z', ascending=False)

# xyz = xyz[xyz.w > 0.5 * 1000]

In [ ]:
plt.scatter(xyz.u, xyz.v)
plt.gca().set_aspect(1)
ths = np.linspace(0, 2*pi, 100)
plt.plot(cfres.rad_curv * np.cos(ths) + cfres.curv_ctr_x, cfres.rad_curv * np.sin(ths) + cfres.curv_ctr_y, 'r')

In [ ]:
cfres

In [ ]:
xi = 0
yi = 0.005
zi = (cfres.beam_int / 1000)# + 0.005
eni = cfres.curv_en_p# + 0.4
# azii = np.arctan2(xyz.iloc[:10].v, xyz.iloc[:10].u).mean()
azii = np.arctan2(-cfres.curv_ctr_y, -cfres.curv_ctr_x) - pi/2
poli = (pi - cfres.scat_ang)# + 2*degrees)
mass_num = 1
charge_num = 1 
bc = np.array(beam_center) / 1000

gas = pytpc.gases.InterpolatedGas('isobutane', 23.2)

tracker = atmc.Tracker(mass_num, charge_num, gas, efield, bfield)
evtgen = atmc.EventGenerator(padplane, vd, 12.5, 280e-9, mass_num, 10.56, 2, tilt, bc)

# this_bfield = np.array([0, 0, 1.8])

In [ ]:
xyz['cx'] = xyz.u - cfres.curv_ctr_x
xyz['cy'] = xyz.v - cfres.curv_ctr_y
xyz['azi'] = np.unwrap(np.arctan2(xyz.cy, xyz.cx) - pi/2)
xyz.azi -= azii

In [ ]:
out = tracker.track_particle(xi, yi, zi, eni, azii, poli)

In [ ]:
orig2 = pd.DataFrame(out, columns=('x', 'y', 'z', 'time', 'en', 'azi', 'pol'))

In [ ]:
plt.plot(orig2.x, orig2.y, 'k', zorder=100)
plt.scatter(xyz.u / 1000, xyz.v / 1000)
plt.gca().set_aspect(1)
ths = np.linspace(0, 2*pi, 100)
# plt.plot(cfres.rad_curv/1000 * np.cos(ths) + cfres.curv_ctr_x/1000, 
#          cfres.rad_curv/1000 * np.sin(ths) + cfres.curv_ctr_y/1000, 'r--', linewidth=1)

In [ ]:
stretch = 1.33

In [ ]:
# plt.plot((orig.z - zi) * stretch + zi, orig.x, 'k-', zorder=10)
plt.plot(orig2.z, orig2.x, 'k', zorder=100)
# plt.plot((xyz.w/1000 - zi) / stretch + zi, xyz.u / 1000, 'c.')
plt.plot(xyz.w / 1000, xyz.u / 1000, 'b.')

# plt.plot((orig.z - zi) * stretch + zi, orig.y, 'k-', zorder=10)
plt.plot(orig2.z, orig2.y, 'k', zorder=100)
plt.plot(xyz.w / 1000, xyz.v / 1000, 'r.')
# plt.plot((xyz.w/1000 - zi) / stretch + zi, xyz.v / 1000, 'y.')

In [ ]:
plt.plot(orig2.z, np.hypot(orig2.x, orig2.y), 'k', zorder=100)
plt.plot(xyz.w/1000, np.hypot(xyz.u, xyz.v)/1000, 'g.')

---
# Minimization here

In [ ]:
minimizer = atmc.Minimizer(tracker, evtgen)

In [ ]:
# real_mesh = np.zeros(512)
# for z, a in raw_xyz[:, 2:4]:
#     real_mesh += transfer(a, 280/80, z) / 0.044

In [ ]:
ctr0 = np.array([xi, yi, zi, eni, azii, poli, 1.75])
sigma = np.array([0, 0, 0, 2.0, 10*degrees, 10*degrees, 0.0])

true_values = xyz[['u', 'v', 'w']].sort_values(by='w', ascending=True).values.copy()
true_values[:, :3] /= 1000
true_ens = xyz.sort_values(by='w', ascending=True).a.values.copy() / 1e6 * 10.56

real_mesh = np.zeros(512)
for z, a in raw_xyz[:, 2:4]:
    real_mesh += transfer(a, 280./80, z)

In [ ]:
from itertools import combinations, repeat, product

In [ ]:
num_iters = [5, 10, 15, 20]
num_pts = [100, 200, 300, 400, 500] #repeat(200)
red_factor = [0.5, 0.6, 0.7, 0.8, 0.9]

In [ ]:
len(list(product(num_iters, num_pts, red_factor))) * 10 * 2

In [ ]:
allctrs = []
for n, p, r in zip(num_iters, num_pts, red_factor):
    ctrs = []
    for i in range(10):
        ctr, min_chis, en_chis, all_params, good_param_idx = \
            minimizer.minimize(ctr0, sigma, true_values, real_mesh, n, p, r, details=True)
        ctrs.append(pd.Series(ctr, index=('x', 'y', 'z', 'en', 'azi', 'pol', 'bmag')))
        
    ctrs = pd.concat(ctrs, axis=1).T
    ctrs['num_iters'] = n
    ctrs['num_pts'] = p
    ctrs['red_factor'] = r

    allctrs.append(ctrs)

In [ ]:
min_chis

In [ ]:
allctrs = pd.concat(allctrs, axis=0, ignore_index=True)

In [ ]:
allctrs.head()

In [ ]:
sns.boxplot(data=allctrs, x='num_pts', y='pol')

In [ ]:
ctr.azi / degrees

In [ ]:
(pi - ctr.pol) / degrees

In [ ]:
plt.plot(real_mesh)

In [ ]:
good_param_idx

In [ ]:
good_params = all_params[good_param_idx.astype('int')]
all_params = all_params.reshape((num_iters, num_pts, 7))

In [ ]:
min_chis

In [ ]:
for v, l in zip(good_params.T, ['x', 'y', 'z', 'en', 'azi', 'pol', 'bmag']):
    plt.plot(v / v[0], label=l)
plt.legend(loc=4)

In [ ]:
plt.subplot(321)
plt.plot(np.ravel(all_params[:, :, 0]), ',')
plt.subplot(322)
plt.plot(np.ravel(all_params[:, :, 1]), ',')
plt.subplot(323)
plt.plot(np.ravel(all_params[:, :, 2]), ',')
plt.hlines(zi, *plt.xlim())
plt.subplot(324)
plt.plot(np.ravel(all_params[:, :, 3]), ',')
plt.hlines(eni, *plt.xlim())
plt.subplot(325)
plt.plot(np.ravel(all_params[:, :, 4]), ',')
plt.hlines(azii, *plt.xlim())
plt.subplot(326)
plt.plot(np.ravel(all_params[:, :, 5]), ',')
plt.hlines(poli, *plt.xlim())

In [ ]:
plt.plot(min_chis, 'o-')
plt.semilogy()

In [ ]:
# ctr = pd.Series(c, index=('x', 'y', 'z', 'en', 'azi', 'pol', 'bmag'))

In [ ]:
res_track = tracker.track_particle(ctr.x, ctr.y, ctr.z, ctr.en, ctr.azi, ctr.pol)

In [ ]:
plt.plot(res_track[:, 0], res_track[:, 1])
# plt.plot(xyz.u / 1000, xyz.v / 1000, '.')
plt.plot(true_values[:, 0], true_values[:, 1], '.')
plt.gca().set_aspect(1)
# plt.axis([-0.275, 0.275, -0.275, 0.275])
# ths = np.linspace(0, 2*pi, 100)
# plt.plot(0.275 * np.cos(ths), 0.275 * np.sin(ths), 'r--', linewidth=1)

In [ ]:
res_track

In [ ]:
plt.plot(res_track[:, 2], res_track[:, 1], 'b')
# plt.plot(xyz.w / 1000, xyz.v / 1000, 'b.')
plt.plot(true_values[:, 2], true_values[:, 1], 'b.')
# plt.plot((xyz.w/1000 - zi) / stretch + zi, xyz.v / 1000, 'b.')
plt.plot(orig2.z, orig2.y, 'b--')

plt.plot(res_track[:, 2], res_track[:, 0], 'r')
# plt.plot(xyz.w / 1000, xyz.u / 1000, 'c.')
plt.plot(true_values[:, 2], true_values[:, 0], 'r.')
# plt.plot((xyz.w/1000 - zi) / stretch + zi, xyz.u / 1000, 'r.')
# plt.plot(orig.z, orig.x, 'r--')

In [ ]:
devs = atmc.mcopt_wrapper.find_deviations(res_track, true_values)
plt.plot(true_values[:, 2], np.abs(devs[:, 0]), 'r.')
plt.plot(true_values[:, 2], np.abs(devs[:, 1]), 'b.')
cx = np.sum(devs**2, -1)**2 / 1e-4
# plt.plot(true_values[:, 2], cx, 'm.')
# plt.semilogy()

In [ ]:
# evtdict = evtgen.make_event(res_pos, res_ens)
# sigs = np.array(list(evtdict.values()))
# badpads = set(excl_pads).union(set(lowg_pads))
# good_items = filter(lambda x: x not in badpads, evtdict)
# res_mesh = np.sum([evtdict[p] for p in good_items], 0)
# res_mesh = np.sum(list(evtdict.values()), 0)
res_mesh = evtgen.make_mesh_signal(res_track[:, :3].copy(), res_track[:, 4].copy())

In [ ]:
plt.plot(real_mesh)
plt.plot(res_mesh)

In [ ]:
endevs = minimizer.find_energy_deviation(res_track[:, :3].copy(), res_track[:, 4].copy(), real_mesh)
plt.plot(endevs)
# plt.plot((res_mesh - real_mesh) / (real_mesh.max() * 0.10))

In [ ]:
endevs = np.round(endevs, decimals=4)

In [ ]:
np.mean(endevs[np.nonzero(endevs)]**2)

In [ ]:
# for z, a in raw_xyz[:, 2:4]:
#     plt.plot(transfer(a, 280/80, z), linewidth=1)

In [ ]:
for v in evtdict.values():
    plt.plot(v, linewidth=1)
# plt.xlim(150, 200)

In [ ]:
plt.plot(list(evtdict.values())[4])

In [ ]:
plt.plot(real_mesh)
plt.plot(res_mesh)
# plt.plot(endevs)

In [ ]:
res_hits = np.zeros(10240)
for p, v in evtdict.items():
    res_hits[p] += v.max()

In [ ]:
pytpc.pad_plot(res_hits, scale='linear', cmap=cmaps.viridis)

In [ ]:
plt.hist(true_values[:, 2], bins=100);
med = np.median(true_values[:, 2])
mad = np.median(np.abs(true_values[:, 2] - med))
plt.vlines((med, med+2*mad, med-2*mad), *plt.ylim())

In [ ]:
zlenTrue = true_values[:, 2].max() - true_values[:, 2].min()
zlenSim = res_track[:, 2].max() - res_track[:, 2].min()

In [ ]:
np.median(np.sum(devs**2, -1)) + (zlenTrue - zlenSim)**2

In [ ]:
plt.plot(res_track[:, 2], np.hypot(res_track[:, 0], res_track[:, 1]))
# plt.plot(xyz.w / 1000, (np.hypot(xyz.u, xyz.v) / 1000), '.')
plt.plot(true_values[:, 2], np.hypot(true_values[:, 0], true_values[:, 1]), '.')
plt.plot(orig2.z, np.hypot(orig2.x, orig2.y), 'r--')

In [ ]:
with sns.axes_style('white'):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter3D(xyz.u/1000, xyz.v/1000, xyz.w/1000)
    ax.plot3D(res_track[:, 0], res_track[:, 1], res_track[:, 2])
#     ax.plot3D(res.x, res.y, res.z, 'r--')
    ax.view_init(45, 90)
    ax.set_aspect(1)